# Extract SourcesTables visit Results in  OGA

 work with Weakly_2023_44
- use jupyter kernel LSST


- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2023/12/26
- update : 2023/12/26


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2023_44/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2023_44/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-7.0.1/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2023_44/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+d053631def 	current w_2023_44 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits

import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
filesource_out="sourceTablevisit_202312.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

## Give the collection

In [11]:
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"

## Config

# Sources

In [12]:
datasetRefs = butler.registry.queryDatasets(datasetType='sourceTable_visit', collections=my_collection, where= "instrument='LATISS'")

In [13]:
all_ref_for_src = []
all_dataId = []
all_src = []
all_exposures = []
all_sourcesTables = []
all_dateobs = []

for i, ref in enumerate(datasetRefs):
    all_ref_for_src.append(ref.dataId)
    print("========================datasetType = sourceTable_visit ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
    the_visit = ref.dataId["visit"]
    the_exposure = the_visit
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    
    # extract time
    raw_img= butler.get('raw', dataId={'exposure': the_visit, 'instrument': 'LATISS', 'detector': 0}, collections = 'LATISS/raw/all')
    meta = raw_img.getMetadata()
    md = meta.toDict()
    date_obs = md['DATE-OBS']
    
    print(the_dataId)
    df_tab=  butler.get(ref,collections=my_collection)
   
    
    
    N=len(df_tab)
    df_tab['visid_idx']= (i*np.ones(N)).astype(int)
    df_tab['dateobs']= (pd.to_datetime(date_obs))
    df_tab['mjd'] = Time(date_obs).mjd
    all_sourcesTables.append(df_tab) 
    all_dataId.append(the_dataId) 
    all_dateobs.append(date_obs)
    
    
 
    
    #registry_query = f"instrument='LATISS' and instrument=0 and visit={the_visit}]"
    #print(f"registry_query = {registry_query}")
    #info = registry.queryDimensionRecords('exposure',where = registry_query)
    
    #if i > 1:
    #    break

========================datasetType = sourceTable_visit ============================================
fullId..................: {band: 'r', instrument: 'LATISS', physical_filter: 'SDSSr_65mm~empty', visit: 2023080300627}
visit...................: 2023080300627
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553/20231120T224314Z
{'day_obs': 20230803, 'seq_num': 627, 'detector': 0}
========================datasetType = sourceTable_visit ============================================
fullId..................: {band: 'i', instrument: 'LATISS', physical_filter: 'empty~SDSSi_65mm', visit: 2023110800359}
visit...................: 2023110800359
band....................: i
physical filter.........: empty~SDSSi_65mm
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553/20231120T224314Z


/tmp/ipykernel_15977/3997895698.py:29: UserWarning: Collections should not be specified with DatasetRef
  df_tab=  butler.get(ref,collections=my_collection)


{'day_obs': 20231108, 'seq_num': 359, 'detector': 0}
========================datasetType = sourceTable_visit ============================================
fullId..................: {band: 'i', instrument: 'LATISS', physical_filter: 'SDSSi_65mm~empty', visit: 2023072000180}
visit...................: 2023072000180
band....................: i
physical filter.........: SDSSi_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553/20231120T224314Z


/tmp/ipykernel_15977/3997895698.py:29: UserWarning: Collections should not be specified with DatasetRef
  df_tab=  butler.get(ref,collections=my_collection)


{'day_obs': 20230720, 'seq_num': 180, 'detector': 0}
========================datasetType = sourceTable_visit ============================================
fullId..................: {band: 'r', instrument: 'LATISS', physical_filter: 'SDSSr_65mm~empty', visit: 2023080100233}
visit...................: 2023080100233
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553/20231120T224314Z


...

/tmp/ipykernel_15977/3997895698.py:29: UserWarning: Collections should not be specified with DatasetRef
  df_tab=  butler.get(ref,collections=my_collection)


{'day_obs': 20230706, 'seq_num': 353, 'detector': 0}
========================datasetType = sourceTable_visit ============================================
fullId..................: {band: 'r', instrument: 'LATISS', physical_filter: 'SDSSr_65mm~empty', visit: 2023070600371}
visit...................: 2023070600371
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553/20231120T224314Z


/tmp/ipykernel_15977/3997895698.py:29: UserWarning: Collections should not be specified with DatasetRef
  df_tab=  butler.get(ref,collections=my_collection)


{'day_obs': 20230706, 'seq_num': 371, 'detector': 0}


/tmp/ipykernel_15977/3997895698.py:29: UserWarning: Collections should not be specified with DatasetRef
  df_tab=  butler.get(ref,collections=my_collection)


In [14]:
len(all_sourcesTables)

1655

In [15]:
all_sourcesTables[0]

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,dec,decl,raErr,decErr,ra_dec_Cov,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,apFlux_35_0_flag,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,apFlux_50_0_flag,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,pixelFlags_streak,pixelFlags_streakCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx,dateobs,mjd
sourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
174585527540908033,270.167336,-25.072239,41624433408,0,100.000000,20.000000,NaN,NaN,270.167336,-25.072239,-25.072239,NaN,NaN,NaN,NaN,NaN,6034.303275,1404.846313,True,2.311594e+04,2852.324978,True,4.657348e+04,4284.495364,True,6.822271e+04,5709.729935,True,9.348015e+04,8103.685755,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,2.020779,219.170964,8.387502e+04,6730.747102,33.703799,31.936875,-1.392675,34.901507,35.268276,-2.087437,NaN,NaN,NaN,8.051163e+04,6565.653429,NaN,25.180333,False,0.04275,False,5.661483e-09,4.638652e-07,-5.714067e-09,4.639164e-07,0.000000,True,True,True,True,False,True,2709.364746,226.706894,True,3712.427272,321.764276,True,NaN,NaN,True,NaN,NaN,True,1314,False,0.915751,0.080252,8.704054,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,1.038563,0.0,561.565125,True,False,True,False,False,True,False,False,False,False,True,False,False,False,113.786469,False,False,False,False,False,False,False,False,False,False,0

In [16]:
df=pd.concat(all_sourcesTables)

In [17]:
df

,coord_ra,coord_dec,ccdVisitId,parentSourceId,x,y,xErr,yErr,ra,dec,decl,raErr,decErr,ra_dec_Cov,calibFlux,calibFluxErr,ap03Flux,ap03FluxErr,ap03Flux_flag,ap06Flux,ap06FluxErr,ap06Flux_flag,ap09Flux,ap09FluxErr,ap09Flux_flag,ap12Flux,ap12FluxErr,ap12Flux_flag,ap17Flux,ap17FluxErr,ap17Flux_flag,ap25Flux,ap25FluxErr,ap25Flux_flag,ap35Flux,ap35FluxErr,ap35Flux_flag,ap50Flux,ap50FluxErr,ap50Flux_flag,ap70Flux,ap70FluxErr,ap70Flux_flag,sky,skyErr,psfFlux,psfFluxErr,ixx,iyy,ixy,ixxPSF,iyyPSF,ixyPSF,ixxDebiasedPSF,iyyDebiasedPSF,ixyDebiasedPSF,gaussianFlux,gaussianFluxErr,extendedness,localPhotoCalib,localPhotoCalib_flag,localPhotoCalibErr,localWcs_flag,localWcs_CDMatrix_2_1,localWcs_CDMatrix_1_1,localWcs_CDMatrix_1_2,localWcs_CDMatrix_2_2,blendedness_abs,blendedness_flag,blendedness_flag_noCentroid,blendedness_flag_noShape,apFlux_12_0_flag,apFlux_12_0_flag_apertureTruncated,apFlux_12_0_flag_sincCoeffsTruncated,apFlux_12_0_instFlux,apFlux_12_0_instFluxErr,apFlux_17_0_flag,apFlux_17_0_instFlux,apFlux_17_0_instFluxErr,apFlux_35_0_flag,apFlux_35_0_instFlux,apFlux_35_0_instFluxErr,apFlux_50_0_flag,apFlux_50_0_instFlux,apFlux_50_0_instFluxErr,extendedness_flag,footprintArea_value,jacobian_flag,jacobian_value,localBackground_instFlux,localBackground_instFluxErr,localBackground_flag,localBackground_flag_noGoodPixels,localBackground_flag_noPsf,pixelFlags_bad,pixelFlags_cr,pixelFlags_crCenter,pixelFlags_edge,pixelFlags_interpolated,pixelFlags_interpolatedCenter,pixelFlags_offimage,pixelFlags_saturated,pixelFlags_saturatedCenter,pixelFlags_suspect,pixelFlags_suspectCenter,pixelFlags_streak,pixelFlags_streakCenter,psfFlux_apCorr,psfFlux_apCorrErr,psfFlux_area,psfFlux_flag,psfFlux_flag_apCorr,psfFlux_flag_edge,psfFlux_flag_noGoodPixels,gaussianFlux_flag,centroid_flag,centroid_flag_almostNoSecondDerivative,centroid_flag_badError,centroid_flag_edge,centroid_flag_noSecondDerivative,centroid_flag_notAtMaximum,centroid_flag_resetToPeak,variance_flag,variance_flag_emptyFootprint,variance_value,calib_astrometry_used,calib_detected,calib_photometry_reserved,calib_photometry_used,calib_psf_candidate,calib_psf_reserved,calib_psf_used,deblend_deblendedAsPsf,deblend_hasStrayFlux,deblend_masked,deblend_nChild,deblend_parentTooBig,deblend_patchedTemplate,deblend_rampedTemplate,deblend_skipped,deblend_tooManyPeaks,hsmPsfMoments_flag,hsmPsfMoments_flag_no_pixels,hsmPsfMoments_flag_not_contained,hsmPsfMoments_flag_parent_source,iDebiasedPSF_flag,iDebiasedPSF_flag_no_pixels,iDebiasedPSF_flag_not_contained,iDebiasedPSF_flag_parent_source,iDebiasedPSF_flag_galsim,iDebiasedPSF_flag_edge,hsmShapeRegauss_flag,hsmShapeRegauss_flag_galsim,hsmShapeRegauss_flag_no_pixels,hsmShapeRegauss_flag_not_contained,hsmShapeRegauss_flag_parent_source,sky_source,detect_isPrimary,visit,detector,band,physical_filter,visid_idx,dateobs,mjd
sourceId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
174585527540908033,270.167336,-25.072239,41624433408,0,100.000000,20.000000,NaN,NaN,270.167336,-25.072239,-25.072239,NaN,NaN,NaN,NaN,NaN,6034.303275,1404.846313,True,2.311594e+04,2852.324978,True,4.657348e+04,4284.495364,True,6.822271e+04,5709.729935,True,9.348015e+04,8103.685755,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,2.020779,219.170964,8.387502e+04,6730.747102,33.703799,31.936875,-1.392675,34.901507,35.268276,-2.087437,NaN,NaN,NaN,8.051163e+04,6565.653429,NaN,25.180333,False,0.042750,False,5.661483e-09,4.638652e-07,-5.714067e-09,4.639164e-07,0.000000,True,True,True,True,False,True,2709.364746,226.706894,True,3712.427272,321.764276,True,NaN,NaN,True,NaN,NaN,True,1314,False,0.915751,0.080252,8.704054,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,1.038563,0.0,561.565125,True,False,True,False,False,True,False,False,False,False,True,False,False,False,113.786469,False,False,False,False,False,False,False,False,False,False,

In [18]:
df.to_csv(filesource_out)

In [19]:
print(df.columns.tolist())

['coord_ra', 'coord_dec', 'ccdVisitId', 'parentSourceId', 'x', 'y', 'xErr', 'yErr', 'ra', 'dec', 'decl', 'raErr', 'decErr', 'ra_dec_Cov', 'calibFlux', 'calibFluxErr', 'ap03Flux', 'ap03FluxErr', 'ap03Flux_flag', 'ap06Flux', 'ap06FluxErr', 'ap06Flux_flag', 'ap09Flux', 'ap09FluxErr', 'ap09Flux_flag', 'ap12Flux', 'ap12FluxErr', 'ap12Flux_flag', 'ap17Flux', 'ap17FluxErr', 'ap17Flux_flag', 'ap25Flux', 'ap25FluxErr', 'ap25Flux_flag', 'ap35Flux', 'ap35FluxErr', 'ap35Flux_flag', 'ap50Flux', 'ap50FluxErr', 'ap50Flux_flag', 'ap70Flux', 'ap70FluxErr', 'ap70Flux_flag', 'sky', 'skyErr', 'psfFlux', 'psfFluxErr', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF', 'ixxDebiasedPSF', 'iyyDebiasedPSF', 'ixyDebiasedPSF', 'gaussianFlux', 'gaussianFluxErr', 'extendedness', 'localPhotoCalib', 'localPhotoCalib_flag', 'localPhotoCalibErr', 'localWcs_flag', 'localWcs_CDMatrix_2_1', 'localWcs_CDMatrix_1_1', 'localWcs_CDMatrix_1_2', 'localWcs_CDMatrix_2_2', 'blendedness_abs', 'blendedness_flag', 'blendedness_flag_

In [20]:
df['dateobs'].unique()

array(['2023-08-04T04:27:26.714000000', '2023-11-09T04:05:42.659000000',
       '2023-07-21T01:15:06.288000000', ...,
       '2023-09-29T00:27:57.787000000', '2023-07-07T02:36:27.194000000',
       '2023-07-07T02:56:33.594000000'], dtype='datetime64[ns]')

In [21]:
df.ra

sourceId
174585527540908033    270.167336
174585527540908034    270.176990
174585527540908035    270.188641
174585527540908036    270.208122
174585527540908037    270.234172
                         ...    
173600090244513988    240.067465
173600090244513989    240.067456
173600090244513990    240.067415
173600090244513991    240.011396
173600090244513992    240.010572
Name: ra, Length: 1356239, dtype: float64